In [31]:
xn = Array([1,2,3])
xo = Array([3,2,1])

temperatures = [1,1,1]

ΔE_vec = xn.^2 .- xo.^2

condition1 = ΔE_vec .< 0
condition2 = .!condition1 .& (rand(3) .< exp.(-ΔE_vec./temperatures))
condition = condition1 .| condition2

println(condition')
println(xn')
println(xo')

println([xn[i] for i in 1:3 if condition[i]])

xo .= xn.*condition .+ xo.*.!condition

Bool[1 1 0]
[1 2 3]
[3 2 1]
[1, 2]


3-element Vector{Int64}:
 1
 2
 1

In [1]:
using ProgressMeter
using Logging
using Base.Threads
using Random #, Distributions

In [ ]:
function metropolis_energy(Nsteps, energy, initial_guess, step_size, gaussian_step=true ,T=1.f0, args...)
    x     = initial_guess
    chain = zeros(Nsteps)
    for i in 1:Nsteps
        if gaussian_step
            x_new = x + step_size*randn()
        else
            x_new = x + step_size*rand()
        end
        ΔE = energy(x_new, args...) - energy(x,args...)
        if ΔE < 0
            x = x_new
        else
            if rand() < exp(-ΔE/T)
                x = x_new
            end
        end
        chain[i] = x
    end
    return x, chain
end;

metropolis_energy (generic function with 3 methods)

In [ ]:
mutable struct PTParams{T}
    TMax       :: T
    TMin       :: T
    λ          :: T
    NTemps     :: Int
    Nexchanges :: Int
    PTParams{T}() where {T} = new()
end;

mutable struct MetropolisParams{T}
    NSteps       :: Int
    StepSize     :: T
    GaussianStep :: Bool
    MetropolisParams{T}() where {T} = new()
end;

In [ ]:
function parallel_tempering(PTParameters::PTParams, MetropolisParams::MetropolisParams, initial_guess)
    
    Ener(x)          = -5*exp(-(x+2)^2) -exp(-2*(x-5.0)^2)

    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    TMin             = PTParameters.TMin
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))

    vectorized = false

    @showprogress for step in 1:PTParameters.Nexchanges+1

        # ADD: check for the lowes energy state of the whole chain
        # this can change the way of computation, as you have to compute the energy at each chain, so 
        # you can save the energy and use that to compute the ΔE

        # Sampling step
        xn = xo .+ StepSize*randn(NTemps)

        if vectorized
            ΔE_vec = Ener.(xn) - Ener.(xo)
            condition1 = ΔE_vec .< 0
            condition2 = .!condition1 .& (rand(NTemps) .< exp.(-ΔE_vec./temperatures))
            condition = condition1 .| condition2
            xo .= xn.*condition .+ xo.*.!condition
        else
            for i in 1:NTemps
                ΔE = Ener(xn[i]) - Ener(xo[i])
                if ΔE < 0
                    xo[i] = xn[i]
                elseif rand() < exp(-ΔE/temperatures[i])
                    xo[i] = xn[i]
                end
            end
        end

        # Exchange step. VECTORIZE ALSO
        for temp in 1:NTemps-1
            ΔE_exchange = (Ener(xo[temp]) - Ener(xo[temp+1])) * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
                # Temp dependency is put inside ΔE_exchange!!!!!!
            elseif rand() < exp(-ΔE_exchange)
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
            end
        end
    end

    return xo
end;

In [ ]:
kind  = Float32

Parms       = PTParams{kind}()
Parms.TMax  = 100
Parms.λ     = 0.92f0
Parms.NTemps     = 100
Parms.Nexchanges = 100

Metrop      = MetropolisParams{kind}();

In [ ]:
parallel_tempering(Parms,Metrop,5.0*randn(Parms.NTemps))

1×100 adjoint(::Vector{Float64}) with eltype Float64:
 100.0  92.0  84.64  77.8688  71.6393  …  0.0307199  0.0282623  0.0260013

100-element Vector{Float64}:
 -1.3030870788827296e16
 -1.0225002614966147e17
  7.346403036055248e16
 -7.579485584178428e15
 -2.1725222341162828e14
  5.1616421587595704e16
 -1.2191810751828277e17
 -1.8642740231644896e16
  1.2612085446780412e16
  1.0360912129496542e17
 -1.824854815000853e16
  1.3134675779727184e16
  1.8100686337672212e16
  ⋮
  6.325729187638064e16
  1.8637192592728228e16
 -5.9640735633027e16
 -4.604688129872133e16
  6.55056781047126e16
  9.626513984770418e16
 -1.7822224360772888e16
  4.7643881338916424e16
 -4.377283029526856e15
 -6.605321573790009e16
 -7.452817728365195e16
  2.5276005244873676e16

LoadError: basta!

In [ ]:


struct Exchanges
    Texchange::Vector
    ExchangeStep::Vector{Int}
end

"""
# Parallel Tempering MCMC (PTMCMC)

with PTParams we define the number of temperatures, the max and min of them and generate a linspace of temperatures. Also we define how many times do we want to exchange the temperatures.
with metropolisparams we can define the number of steps that we want to do in each time we sample from metropolis (aka the steps before an exchange)

Should I use dictionaries as inputs instead of structs? maybe dicts are more flexible...

"""

function parallel_tempering(exchanges, PTParameters::PTParams, MetropolisParams::MetropolisParams, f, initial_guess,args...)
    NTemps       = PTParameters.NTemps
    Nexchanges   = PTParameters.Nexchanges
    TMax         = PTParameters.TMax
    TMin         = PTParameters.TMin
    NSteps       = MetropolisParams.NSteps
    StepSize     = MetropolisParams.StepSize
    GaussianStep = MetropolisParams.GaussianStep
    # temperatures = range(TMin, TMax, length=NTemps)
    # TN=T0*λ^N
    # TODO: we can have an arbitrary way of choosing the temperatures, Let the user decide.
    
    λ               = PTParameters.λ
    temperatures    = zeros(NTemps)
    temperatures[1] = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]
    
        display(temperatures)
        error("aki")
    
    #temperatures = TMax * (TMin/TMax).^(range(0, NTemps-1, length=NTemps))
    # n = range(0, NTemps-1, length=NTemps)/(NTemps-1)
    # temperatures = TMax.^(1 .-n).*TMin.^n
    plot(temperatures)
    title("Tmin=$(minimum(temperatures)) Tmax=$TMax")
    # logscale in y axis
    yscale("log")
    savefig("out/temperatures.png")
    clf()
    x = initial_guess*ones(NTemps)
    # x = x .+ randn(NTemps)
    d = Normal(0,1)
    x = x .+ rand(d, NTemps)
    @info "Temperatures: $temperatures"
    @info "Initial guess: $x"
    total_chains = zeros(Nexchanges+1,NTemps, NSteps)
    total_chains[1,:,1] = x
    
    count = 0
    
    @showprogress for step in 1:PTParameters.Nexchanges+1
        # Sampling step
        Threads.@threads for i in 1:NTemps # parallel!!!!!!!

            # TODO: can i change step characteristics to explore the space better?
            # propose a state and metropolis accept reject. split into step choosing and metropolis accepting. Thought only to have energies, 
            # so everything is an exponential
            x[i], total_chains[step ,i, :] = metropolis_energy(NSteps, f, x[i], StepSize, GaussianStep, temperatures[i], args...)
        end
        # Exchange step
        for i in 1:NTemps-1 # starting from highest temperature, going to lowest
            # exchange for temperature i and i-1
            ΔE_exchange = (f(total_chains[step, i, end], args...) - f(total_chains[step, i+1, end], args...)) * (1/temperatures[i] - 1/temperatures[i+1])
            ΔE_exchange = -f(total_chains[step, i, end], args...)/temperatures[i+1] - f(total_chains[step, i+1, end], args...)/temperatures[i]+f(total_chains[step, i, end], args...)/temperatures[i] + f(total_chains[step, i+1, end], args...)/temperatures[i+1]
            if ΔE_exchange < 0
                x[i], x[i+1] = x[i+1], x[i]
                count += 1
            elseif rand() < exp(-ΔE_exchange)
                x[i], x[i+1] = x[i+1], x[i]
            end
        end
    end
    print(count)
    return total_chains[end,end,end], total_chains
end

function flatten_chains(chains, temp_idx)
    temp_chain = []
    for j in 1:size(chains,1)
        append!(temp_chain, chains[j,temp_idx,:])
    end
    return temp_chain[2:end]
end

gaussian_energy_landscape(x,args...) = -5*exp(-(x+2)^2) -exp(-2*(x-5.0)^2)
plot_landscapes = true
using PyPlot
clf()
if plot_landscapes
    x = range(-10, 10, length=1000)
    plot(x, gaussian_energy_landscape.(x))
    # plot a vertical line in the minimum
    axhline(y=minimum(gaussian_energy_landscape.(x)), color="red", linestyle="--")
    savefig("out/energy_landscape.png")
    clf()
end

#lets find the min with parallel tempering

initial_guess = -1.0
λ = 0.7
TMax = 100.0
TMin = TMax*λ
NTemps = 50
Nexchanges = 100
NSteps = 100
StepSize = 0.01
GaussianStep = true

PTParameters = PTParams(TMax, TMin, NTemps, Nexchanges)
MetropolisParameters = MetropolisParams(NSteps, StepSize, GaussianStep)

exchanges = Exchanges([], [])

quadratic_energy_landscape(x,args...) = x^2

final, chains = parallel_tempering(exchanges, PTParameters, MetropolisParameters, gaussian_energy_landscape, initial_guess)

chains1 = flatten_chains(chains, 1)


# for i in 1:NTemps
#     plot(chains[:,i,:], label="T = $(round(range(TMax, TMin, length=NTemps)[i], digits=2))")
# end

# plot(chains1, label="T = $(round(range(TMax, TMin, length=NTemps)[1], digits=2))")

# chains2 = flatten_chains(chains, 2)
# plot(chains2, label="T = $(round(range(TMax, TMin, length=NTemps)[2], digits=2))")

# chains3 = flatten_chains(chains, 3)
# plot(chains3, label="T = $(round(range(TMax, TMin, length=NTemps)[3], digits=2))")

# lowest_temp_chain = flatten_chains(chains, NTemps)
# sec_lowest_temp_chain = flatten_chains(chains, NTemps-1)
# plot(sec_lowest_temp_chain[size(sec_lowest_temp_chain,1)-2*NSteps:end], label="T = $(round(range(TMax, TMin, length=NTemps)[NTemps-1], digits=2))")

# plot(lowest_temp_chain[size(lowest_temp_chain,1)-2*NSteps:end], label="T = $(round(range(TMax, TMin, length=NTemps)[NTemps], digits=2))")

# for i in NTemps-2:NTemps
temperatures = TMax * (TMin/TMax).^(range(0, NTemps-1, length=NTemps))

temp_labels = round.(temperatures, digits=2)

for i in NTemps-1:NTemps
    plot(flatten_chains(chains, i), label="T = $(temp_labels[i])")
end

title("$final")
legend()
savefig("out/chain.png")
close(io)

# using PyCall
# @pyimport matplotlib.animation as anim
# using PyPlot


# fig, ax = PyPlot.subplots(nrows=1, ncols=1, figsize=(7, 2.5))
# # ax1, ax2 = axes

# low_T_chain = flatten_chains(chains, NTemps)

# println(size(chains))

# println(size(low_T_chain))

# println()


# function make_frame(i)
#     ax.clear()
#     ax.set_title("$(i+1)")
#     ax.plot(x, gaussian_energy_landscape.(x))
#     ax.plot(low_T_chain[i+1], gaussian_energy_landscape(low_T_chain[i+1]), "ro")
    
#     # ax1.clear()
#     # ax2.clear()
#     # ax1.imshow(A[:,:,i+1, 1])
#     # ax2.imshow(A[:,:,i+1, 2])
# end

# N_iter_per_temp = size(low_T_chain,1)

# frames = [N_iter_per_temp-200:N_iter_per_temp-1]

# myanim = anim.FuncAnimation(fig, make_frame, frames=frames, interval=20, blit=false)
# myanim[:save]("test2.gif", bitrate=-1)